# 3 Tile tract and patch lists

In this notebook we will generate the list of patches that are present on a given VISTA tile. We will do so such that the tile is completely covered. That is we aim to provide every patch that contains any region of the tile.

In [7]:
from astropy.table import Table, Column, vstack # Perhaps we should use LSST tables

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
VIDEO_IN = './data/video_images_overview_20200820.csv'
VHS_IN = './data/vhs_images_overview_20201102.csv'

In [9]:
video_ims = Table.read(VIDEO_IN)
vhs_ims = Table.read(VHS_IN)

In [10]:
len(vhs_ims),len(video_ims)

(887732, 137269)

In [11]:
def fileToType(filename):
    filetype = ''
    types = {
        'tile':'_tl.fit',
        'stack':'_st.fit',
    }
    for k,v in types.items():
        #print(k,v)
        if filename.endswith(v):
            filetype = k
       
    return filetype
video_ims['type'] = [fileToType(f) for f in video_ims['file']]
vhs_ims['type'] = [fileToType(f) for f in vhs_ims['file']]

In [12]:
video_ims = video_ims[video_ims['type']=='tile']
vhs_ims = vhs_ims[vhs_ims['type']=='tile']

In [13]:
len(vhs_ims),len(video_ims)

(33971, 2205)

In [14]:
#We are using a rings skymap
from lsst.geom import SpherePoint 
from lsst.geom import degrees
from lsst.skymap.ringsSkyMap import RingsSkyMap, RingsSkyMapConfig

#Following taken from dmu1/2_Survey_comparisons.ipynb test choice
coord = SpherePoint(35.429025*degrees,-4.90853*degrees)
config = RingsSkyMapConfig()
#These config options are chose to be the same as HSC:
#https://github.com/lsst/obs_subaru/blob/master/config/hsc/makeSkyMap.py
#and copied for obs_vista
#https://github.com/lsst-uk/obs_vista/blob/master/config/makeSkyMap.py
config.numRings = 120
config.projection = "TAN"
config.tractOverlap = 1.0/60 # Overlap between tracts (degrees)
config.pixelScale = 0.168
sm = RingsSkyMap(config)
sm.findTract(coord)

TractInfo(id=8524, ctrCoord=[0.8087076683756161, 0.5811684769959703, -0.09074749834931431])

In [15]:
## Test on know tile

In [16]:
#Tile from dmu4/dmu4_Example
EX_TILE = "20121122/v20121122_00088_st_tl.fit"
ex_row = video_ims[
    video_ims['file'] == "/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/"+EX_TILE
][0]
ex_row

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type
str98,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str5
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20121122/v20121122_00088_st_tl.fit,35.43265,-4.72313,36.1259211592099,36.13048720302681,34.91284867786413,34.914758535953574,-4.074792762244068,-5.561089113115699,-4.077214438128979,-5.563516534082509,Ks,216357120,95f818d1b99dcee749b276666480cdc6,tile


In [17]:
ex_row['ra_0_0']

36.1259211592099

In [18]:
patches = sm.findTractPatchList(
        [
            SpherePoint(ex_row['ra_0_0']*degrees,ex_row['dec_0_0']*degrees),
            SpherePoint(ex_row['ra_0_y']*degrees,ex_row['dec_0_y']*degrees),
            SpherePoint(ex_row['ra_x_0']*degrees,ex_row['dec_x_0']*degrees),
            SpherePoint(ex_row['ra_x_y']*degrees,ex_row['dec_x_y']*degrees)
        ]
    )
t = patches[0][0]

In [19]:
patches

[(TractInfo(id=8766, ctrCoord=[0.8103482053641914, 0.5823474300594684, -0.06486336998774297]),
  (PatchInfo(index=(2, 0), innerBBox=(minimum=(8000, 0), maximum=(11999, 3999)), outerBBox=(minimum=(7900, 0), maximum=(12099, 4099))),
   PatchInfo(index=(2, 1), innerBBox=(minimum=(8000, 4000), maximum=(11999, 7999)), outerBBox=(minimum=(7900, 3900), maximum=(12099, 8099))),
   PatchInfo(index=(2, 2), innerBBox=(minimum=(8000, 8000), maximum=(11999, 11999)), outerBBox=(minimum=(7900, 7900), maximum=(12099, 12099))),
   PatchInfo(index=(3, 0), innerBBox=(minimum=(12000, 0), maximum=(15999, 3999)), outerBBox=(minimum=(11900, 0), maximum=(16099, 4099))),
   PatchInfo(index=(3, 1), innerBBox=(minimum=(12000, 4000), maximum=(15999, 7999)), outerBBox=(minimum=(11900, 3900), maximum=(16099, 8099))),
   PatchInfo(index=(3, 2), innerBBox=(minimum=(12000, 8000), maximum=(15999, 11999)), outerBBox=(minimum=(11900, 7900), maximum=(16099, 12099))),
   PatchInfo(index=(4, 0), innerBBox=(minimum=(16000, 0

In [20]:
import json
tp_dict ={}
for tract in patches:
    #print(tract[0].getId())
    #tp_dict[tract[0].getId()] = [[t.getIndex()[0],t.getIndex()[1]] for t in tract[1]]
    tp_dict[int(tract[0].getId())] = [t.getIndex() for t in tract[1]]

j = json.dumps(tp_dict, separators=(',', ':')) 
j

'{"8766":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"8524":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],"8765":[[0,0],[0,1],[0,2]],"8523":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}'

In [21]:
def corners_to_patch_list(ex_row):
    """Take a ra dec limited region and return a list of all patches contained witing it
    
    Inputs
    =======
    ex_row    astropy.table.row
              Row of image overview table containing corner columns
    
    Returns
    =======
    j        str
             String which can be loaded by json to create a dictionary
             the tracts indices are strings as required by json
    """
    try:
        patches = sm.findTractPatchList(
            [
                SpherePoint(ex_row['ra_0_0']*degrees,ex_row['dec_0_0']*degrees),
                SpherePoint(ex_row['ra_0_y']*degrees,ex_row['dec_0_y']*degrees),
                SpherePoint(ex_row['ra_x_0']*degrees,ex_row['dec_x_0']*degrees),
                SpherePoint(ex_row['ra_x_y']*degrees,ex_row['dec_x_y']*degrees)
            ]
        )
    
        tp_dict ={}
        for tract in patches:
            tp_dict[int(tract[0].getId())] = [t.getIndex() for t in tract[1]]
        j = json.dumps(tp_dict, separators=(',', ':')) 
    except:
        #print(ex_row['file']+" failed")
        j=''
    return j
corners_to_patch_list(video_ims[0])

'{"8766":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"8524":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],"8765":[[0,0],[0,1],[0,2]],"8523":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}'

In [22]:
video_ims['tract_patch_json'] = [corners_to_patch_list(row) for row in video_ims]
vhs_ims['tract_patch_json'] = [corners_to_patch_list(row) for row in vhs_ims]
#corners_to_patch_list(video_ims[0])

In [23]:
video_ims[:5]

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type,tract_patch_json
str98,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str5,str631
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00444_st_tl.fit,35.429542,-4.72215,36.12993723683939,36.13433423332165,34.912894878395214,34.91462725348546,-4.072782101046067,-5.559458172880838,-4.07547203760631,-5.562153562881393,Y,148561920,ccef567359a9d3f1cef197b5b5dad958,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00492_st_tl.fit,52.63885,-27.48791,53.376808661281494,51.6905687690388,53.363677671568034,51.69601464898924,-28.176115497300387,-28.180884566723634,-26.96066324597599,-26.965386169492707,J,174971520,68fe6e1c16640525f9137e139ae269a6,tile,"{""4849"":[[6,8],[7,8],[8,8]],""5063"":[[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[4,0],[4,1],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[5,0],[5,1],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[6,0],[6,1],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[7,0],[7,1],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[8,0],[8,1],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7]],""4848"":[[0,8],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8]],""5062"":[[0,0],[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[1,0],[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[2,0],[2,1],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00396_st_tl.fit,35.428004,-4.7219,36.130230295324694,36.134884600391395,34.913103896675054,34.91508881101303,-4.071964313054827,-5.561119342013037,-4.074296412803666,-5.5634573489486225,Y,146859840,2b3c420a7d15853f89f53c893ef9526e,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20141029/v20141029_00364_st_tl.fit,52.637771,-27.48048,53.37745012628302,51.68913871643598,53.36462194039669,51.69488106361268,-28.174285005665155,-28.178915893328604,-26.960698293435776,-26.965286581454887,Y,152732160,77a38c51b547fca022c539ea718a47bf,tile,"{""4849"":[[6,8],[7,8],[8,8]],""5063"":[[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[4,0],[4,1],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[5,0],[5,1],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[6,0],[6,1],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[7,0],[7,1],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[8,0],[8,1],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7]],""4848"":[[0,8],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8]],""5062"":[[0,0],[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[1,0],[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[2,0],[2,1],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20141029/v20141029_00316_st_tl.fit,35.437767,-4.72747,36.13052006717623,36.13523423715193,34.912898410633574,34.914941744138545,-4.072033411697982,-5.561286819567744,-4.074791337249168,-5.564050760740607,Y,154330560,6a8a3cfacd2

In [24]:
vhs_ims[-5:]

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,visit,exptime,hash,type,tract_patch_json
str96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,int64,int64,str32,str5,str1366
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20150101/v20150101_00369_st_tl.fit,168.272433,-28.53732,169.05437636993554,169.06631099308342,167.6869086670543,167.67951650142442,-27.70666474383839,-29.190602673273954,-27.707517771907426,-29.191478567874846,J,154681920,0,0,8142bfe05ebae11cad401166d2e84195,tile,"{""5133"":[[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""4917"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8]],""4918"":[[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""5132"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20150101/v20150101_00405_st_tl.fit,130.02935,-28.53725,130.81106966927578,130.8227028665792,129.443607418477,129.4359126895467,-27.705955591324074,-29.19001340524193,-27.707293104216927,-29.191370743650776,Ks,200427840,0,0,cec11588c4b57794a0e00d5b1e428cc2,tile,"{""5110"":[[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""4895"":[[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""5109"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3],[4,0],[4,1],[4,2],[4,3]],""4894"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20150101/v20150101_00183_st_tl.fit,67.731033,-28.53732,68.51320107943886,68.52477503690967,67.14554884016694,67.1378006311929,-27.706989827379495,-29.190391071164463,-27.707291041270878,-29.190711544478518,J,196289280,0,0,dcdd54940781b4bc43a20d927f07d255,tile,"{""5072"":[[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3],[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""4858"":[[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""5071"":[[0,0],[0,1],[0,2],[0,3]],""4857"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20150101/v20150101_00489_st_tl.fit,133.730333,-28.53732,134.51188985410403,134.52362303718797,133.1444926860111,133.13690087537915,-27.706200455581417,-29.190100228536224,-27.70733066820755,-29.191251287092015,J,159482880,0,0,d62d720bb9b20b8037264f7b786f92c0,tile,"{""5112"":[[3,0],[3,1],[3,2],[3,3],[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""4897"":[[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""5111"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]],""4896"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20150101/v20150101_00381_st_tl.fit,169.50605,-28.53725,170.28790528843197,170.29950872768873,168.92050038814742,168.91277539324332,-27.7063709249

In [18]:
video_ims[6]['tract_patch_json']

'{"3053":[[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]],"3054":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"2876":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]],"2877":[[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]]}'

In [25]:
#video_ims.write(VIDEO_IN.replace('images','tiles_tracts_patches'),overwrite=True)
vhs_ims.write(VHS_IN.replace('images','tiles_tracts_patches'),overwrite=True)

In [4]:
video_ims = Table.read(VIDEO_IN.replace('images','tiles_tracts_patches'))

In [39]:
video_ims.add_column(Column(
    data= [t.split('/')[-2] for t in video_ims['file']],
    name='date'))

In [42]:
tract = 8524
patch = [3,5]

#has_tract_patch = (
#    [str(tract) in j for j in video_ims['tract_patch_json']]
#    and [patch in j for j in video_ims['tract_patch_json']]
#)

In [7]:
def has_patch(tract, patch, j):
    """Take a json string and return true if a given patch is in it
    """
    j = json.loads(j)
   
    try:
        has_patch = patch in j[str(tract)]
    except KeyError:
        has_patch = False
    return has_patch

In [44]:
import json
j = json.loads(video_ims[6]['tract_patch_json'])

In [45]:
mask = [has_patch(tract, patch, j) for j in video_ims['tract_patch_json']]
mask &= video_ims['filter'] =='Y'
np.sum(mask)

218

In [46]:
len(np.unique(video_ims[mask]['date']))

119

In [56]:
s = "makeCoaddTempExp.py data --rerun coadd"
for date in np.unique(video_ims[mask]['date'])[:5]:
    d = date[0:4]+'-'+date[4:6]+'-'+date[6:9]#+'^'
    s+=" --selectId filter=VISTA-Y dateObs={} ".format(d)
s + "--id filter=VISTA-Y tract=8524 patch=3,5 --clobber-config"

'makeCoaddTempExp.py data --rerun coadd --selectId filter=VISTA-Y dateObs=2009-11-07  --selectId filter=VISTA-Y dateObs=2009-11-08  --selectId filter=VISTA-Y dateObs=2009-11-10  --selectId filter=VISTA-Y dateObs=2009-11-11  --selectId filter=VISTA-Y dateObs=2009-11-12 --id filter=VISTA-Y tract=8524 patch=3,5 --clobber-config'

In [50]:
video_ims[mask]

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type,tract_patch_json,date
str93,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str4,str631,str8
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00444_st_tl.fit,35.429542,-4.72215,36.12993723683939,36.13433423332165,34.912894878395214,34.91462725348546,-4.072782101046067,-5.559458172880838,-4.07547203760631,-5.562153562881393,Y,148561920,ccef567359a9d3f1cef197b5b5dad958,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}",20140924
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00396_st_tl.fit,35.428004,-4.7219,36.130230295324694,36.134884600391395,34.913103896675054,34.91508881101303,-4.071964313054827,-5.561119342013037,-4.074296412803666,-5.5634573489486225,Y,146859840,2b3c420a7d15853f89f53c893ef9526e,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}",20140924
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20141029/v20141029_00316_st_tl.fit,35.437767,-4.72747,36.13052006717623,36.13523423715193,34.912898410633574,34.914941744138545,-4.072033411697982,-5.561286819567744,-4.074791337249168,-5.564050760740607,Y,154330560,6a8a3cfacd21ab9bba56c2cb69b6c6d2,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}",20141029
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20091117/v20091117_00064_st_tl.fit,36.482612,-4.64051,37.18214668914269,37.18455114266433,35.961094380954876,35.960860616191255,-3.986773055171546,-5.4800244567727505,-3.9871165813329927,-5.480369886679151,Y,158469120,7a7a06a1468331a60c215004ea963fe4,tile,"{""8767"":[[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""8525"":[[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8766"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]],""8524"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]]}",20091117
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20091117/v20091117_00118_st_tl.fit,36.482412,-4.64021,37.18439590587843,37.18664987059711,35.960593173902,35.96021107900822,-3.989497273096471,-5.478072627426385,-3.989740744137168,-5.47831774407417,Y,158964480,cabff351a9f6f4641cf4ab0ce0415ff6,

In [12]:
has_patch(8766, [2,0], video_ims['tract_patch_json'][0])

True

In [38]:
video_ims.colnames

['file',
 'ra',
 'dec',
 'ra_0_0',
 'ra_0_y',
 'ra_x_0',
 'ra_x_y',
 'dec_0_0',
 'dec_0_y',
 'dec_x_0',
 'dec_x_y',
 'filter',
 'size',
 'hash',
 'type',
 'tract_patch_json']

In [34]:
patch

'3,5'

In [6]:
video_ims[video_ims['filter'] == 'Z']

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type,tract_patch_json
str93,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str4,str631
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20111120/v20111120_00210_st_tl.fit,36.482817,-4.63425,37.18231090569302,37.18652006406991,35.961664488735046,35.963238113713345,-3.986769532545322,-5.478937149811594,-3.9886036193652252,-5.480776307213548,Z,138787200,23833eda3e2c165fa7fa9782e8429575,tile,"{""8767"":[[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""8525"":[[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8766"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]],""8524"":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20111120/v20111120_00150_st_tl.fit,36.475379,-4.63366,37.18278858342001,37.18683108393643,35.96030105038202,35.96170142387394,-3.987258983515388,-5.480578803996909,-3.989021313266808,-5.482345914705389,Z,140713920,34d3f16914ac92a2a6e945bd07291d53,tile,"{""8767"":[[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""8525"":[[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8766"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]],""8524"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20111025/v20111025_00443_st_tl.fit,36.475237,-4.64066,37.1800139329064,37.184436469525174,35.9598402011581,35.961624455456835,-3.986565736311755,-5.480625614035888,-3.9884329935991962,-5.482498312600167,Z,137851200,18a534efcc4b6c15d5d488e4221d7b52,tile,"{""8767"":[[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""8525"":[[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8766"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]],""8524"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20111025/v20111025_00383_st_tl.fit,36.476708,-4.63325,37.181410618264934,37.18578453416028,35.9614139082408,35.96315111857923,-3.9866854531233167,-5.480129939790323,-3.988549620450015,-5.481999463727386,Z,137928960,99f2e17ce9c706dbd2b9efb7b9f92e12,tile,"{""8767"":[[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3],[6,0],[6,1],[6,2],[6,3],[7,0],[7,1],[7,2],[7,3],[8,0],[8,1],[8,2],[8,3]],""8525"":[[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8766"":[[0,0],[0,1],[0,2],[0,3],[1,0],[1,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3],[3,0],[3,1],[3,2],[3,3]],""8524"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3

In [18]:
has_8524_33 = np.array([has_patch(8524, [3,3], j) for j in video_ims['tract_patch_json']])
has_8524_55 = np.array([has_patch(8524, [5,5], j) for j in video_ims['tract_patch_json']])

In [19]:
np.sum(has_8524_33 & has_8524_55)

234

In [24]:
np.unique(video_ims[has_8524_55  ]['filter'])

H
J
Ks
Y


## 2. Make total tract_patch_json dicts

In [28]:
video_total_patch_dict = {}
vhs_total_patch_dict = {}

for tile in video_ims[video_ims['type']=='tile']:
    tile_dict = json.loads(tile['tract_patch_json'])
    for tract in tile_dict:
        patches = set([str(p) for p in tile_dict[tract]])
        try:
            video_total_patch_dict[tract] = set(video_total_patch_dict[tract]).union(patches)
            
        except KeyError:
            video_total_patch_dict[tract] = patches
            
for tile in vhs_ims[vhs_ims['type']=='tile']:
    tile_dict = json.loads(tile['tract_patch_json'])
    for tract in tile_dict:
        patches = set([str(p) for p in tile_dict[tract]])
        try:
            vhs_total_patch_dict[tract] = set(vhs_total_patch_dict[tract]).union(patches)
            
        except KeyError:
            vhs_total_patch_dict[tract] = patches

In [29]:
vhs_total_patch_dict

{'7839': {'[0, 0]',
  '[0, 1]',
  '[0, 2]',
  '[0, 3]',
  '[0, 4]',
  '[0, 5]',
  '[0, 6]',
  '[0, 7]',
  '[0, 8]',
  '[1, 0]',
  '[1, 1]',
  '[1, 2]',
  '[1, 3]',
  '[1, 4]',
  '[1, 5]',
  '[1, 6]',
  '[1, 7]',
  '[1, 8]',
  '[2, 0]',
  '[2, 1]',
  '[2, 2]',
  '[2, 3]',
  '[2, 4]',
  '[2, 5]',
  '[2, 6]',
  '[2, 7]',
  '[2, 8]',
  '[3, 0]',
  '[3, 1]',
  '[3, 2]',
  '[3, 3]',
  '[3, 4]',
  '[3, 5]',
  '[3, 6]',
  '[3, 7]',
  '[3, 8]',
  '[4, 0]',
  '[4, 1]',
  '[4, 2]',
  '[4, 3]',
  '[4, 4]',
  '[4, 5]',
  '[4, 6]',
  '[4, 7]',
  '[4, 8]',
  '[5, 0]',
  '[5, 1]',
  '[5, 2]',
  '[5, 3]',
  '[5, 4]',
  '[5, 5]',
  '[5, 6]',
  '[5, 7]',
  '[5, 8]',
  '[6, 0]',
  '[6, 1]',
  '[6, 2]',
  '[6, 3]',
  '[6, 4]',
  '[6, 5]',
  '[6, 6]',
  '[6, 7]',
  '[6, 8]',
  '[7, 0]',
  '[7, 1]',
  '[7, 2]',
  '[7, 3]',
  '[7, 4]',
  '[7, 5]',
  '[7, 6]',
  '[7, 7]',
  '[7, 8]',
  '[8, 0]',
  '[8, 1]',
  '[8, 2]',
  '[8, 3]',
  '[8, 4]',
  '[8, 5]',
  '[8, 6]',
  '[8, 7]',
  '[8, 8]'},
 '7600': {'[0, 0]',

In [30]:
n = 0
for tract in vhs_total_patch_dict:
    n += len(vhs_total_patch_dict[tract])

In [31]:
n

670137